# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why?


The target is discrete, binary specificially. The only outcomes are pass or fail. Therefore this is a classification problem.

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [42]:
# Import libraries
import numpy as np
import pandas as pd

In [43]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
# Note: The last column 'passed' is the target/label, all other are feature columns

#seed = 1234 # pick global seed for reproducability
seed = 0

Student data read successfully!


Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [44]:
# TODO: Compute desired values - replace each '?' with an appropriate expression/function call
n_students = student_data.shape[0]
n_features = student_data.shape[1] - 1
n_passed = [x == "yes" for x in student_data["passed"]].count(True)
n_failed = [x == "yes" for x in student_data["passed"]].count(False)
grad_rate = float(n_passed) / (n_passed + n_failed)
print "Total number of students: {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features: {}".format(n_features)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 30
Graduation rate of the class: 0.67%


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [45]:
# Extract feature (X) and target (y) columns
#print student_data.columns
#print type(student_data.columns)
#print student_data.index
#print type(student_data.index)
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print "Feature column(s):-\n{}".format(feature_cols)
print "Target column: {}".format(target_col)

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
print "\nFeature values:-"
print X_all.head()  # print the first 5 rows

Feature column(s):-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: passed

Feature values:-
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [46]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
print "Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [47]:
# First, decide how many training vs test samples you want
num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train

# TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset
from sklearn import cross_validation
X_train,X_test,y_train,y_test = cross_validation.train_test_split(X_all,y_all,
                                                                  train_size=300,
                                                                  random_state=seed)

print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])
# Note: If you need a validation set, extract it from within training data

Training set: 300 samples
Test set: 95 samples


## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What is the theoretical O(n) time & space complexity in terms of input size?
- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

-----

### My choices

1. Decision Tree 
2. Support Vector Machines
3. Naive Bayes

Ensemble method Random Forests is added to see if it might improve on Decision Tree results.

### Decision Tree

Time Complexity:

> O(F N log(N))

Space Complexity:

> O(sqrt(F) N log(N))

N = number of training examples,
F = number of features

Some advantages of decision trees are*:
- Simple to understand and to interpret. Trees can be visualised.
- Requires little data preparation. Other techniques often require data normalisation, dummy variables need to be created and blank values to be removed. Note however that this module does not support missing values.
- The cost of using the tree (i.e., predicting data) is logarithmic in the number of data points used to train the tree.
- Able to handle both numerical and categorical data. Other techniques are usually specialised in analysing datasets that have only one type of variable. See algorithms for more information.
- Able to handle multi-output problems.
- Uses a white box model. If a given situation is observable in a model, the explanation for the condition is easily explained by boolean logic. By contrast, in a black box model (e.g., in an artificial neural network), results may be more difficult to interpret.
- Possible to validate a model using statistical tests. That makes it possible to account for the reliability of the model.
- Performs well even if its assumptions are somewhat violated by the true model from which the data were generated.

The disadvantages of decision trees include*:
- Decision-tree learners can create over-complex trees that do not generalise the data well. This is called overfitting. Mechanisms such as pruning (not currently supported), setting the minimum number of samples required at a leaf node or setting the maximum depth of the tree are necessary to avoid this problem.
- Decision trees can be unstable because small variations in the data might result in a completely different tree being generated. This problem is mitigated by using decision trees within an ensemble.
- The problem of learning an optimal decision tree is known to be NP-complete under several aspects of optimality and even for simple concepts. Consequently, practical decision-tree learning algorithms are based on heuristic algorithms such as the greedy algorithm where locally optimal decisions are made at each node. Such algorithms cannot guarantee to return the globally optimal decision tree. This can be mitigated by training multiple trees in an ensemble learner, where the features and samples are randomly sampled with replacement.
- There are concepts that are hard to learn because decision trees do not express them easily, such as XOR, parity or multiplexer problems.
- Decision tree learners create biased trees if some classes dominate. It is therefore recommended to balance the dataset prior to fitting with the decision tree.

My choice because:

Results are easy to interpret due to white box nature of model. This will help when explaining to school board. Another advantage is the low computational complexity.

Results:

Training Set Size | 100 | 200 | 300
------:|------:|------:|------:|
Training (usec) | 864 | 1128 | 1686
Prediction Train (usec) | 339 | 168 | 224
F1 Train | 1.0 | 1.0 | 1.0
Prediction Test (usec) | 152 | 148 | 143
F1 Test | 0.6942 | 0.7131 | 0.7166


### SVM

Time Complexity (RBF kernel using SMO solver):

> O(N^2 F)

Time Complexity (linear kernel):

> O(N F) 

Space Complexity:

> O(N^2)

N = number of training examples,
F = number of features

Some advantages of support vector machines are*:
- Effective in high dimensional spaces.
- Still effective in cases where number of dimensions is greater than the number of samples.
- Uses a subset of training points in the decision function (called support vectors), so it is also memory efficient.
- Versatile: different Kernel functions can be specified for the decision function. Common kernels are provided, but it is also possible to specify custom kernels.

The disadvantages of support vector machines include*:
- If the number of features is much greater than the number of samples, the method is likely to give poor performances.
- SVMs do not directly provide probability estimates, these are calculated using an expensive five-fold cross-validation (see Scores and probabilities, below).
- Black box. Can't interpret results.

My choice because:

Offers higher accuracy typically. Might consider if accuracy is signifcantly better. Drawbacks are higher computational complexity and black box nature of model.

Results:

Training Set Size | 100 | 200 | 300
------:|------:|------:|------:|
Training (usec) | 944 | 3637 | 5746
Prediction Train (usec) | 643 | 1981 | 4002
F1 Train | 0.8591 | 0.8693 | 0.8692
Prediction Test (usec) | 632 | 1000 | 2364
F1 Test | 0.7838 | 0.7755 | 0.7586


### Naive Bayes

Time Complexity:

> O(N)

Space Complexity:

> O(N)

N = number of training examples,
F = number of features

Some advantages of naive bayes include*:
- In spite of their apparently over-simplified assumptions, naive Bayes classifiers have worked quite well in many real-world situations, famously document classification and spam filtering. They require a small amount of training data to estimate the necessary parameters. (For theoretical reasons why naive Bayes works well, and on which types of data it does, see the references below.)
- Naive Bayes learners and classifiers can be extremely fast compared to more sophisticated methods. The decoupling of the class conditional feature distributions means that each distribution can be independently estimated as a one dimensional distribution. This in turn helps to alleviate problems stemming from the curse of dimensionality.

The disadvantages of naive bayes include*:
- On the flip side, although naive Bayes is known as a decent classifier, it is known to be a bad estimator, so the probability outputs from predict_proba are not to be taken too seriously.

My choice because:

One of the easiest to use models. Does not require any parameters. Fast model. Can give good results depending on domain.

Results:

Training Set Size | 100 | 200 | 300
------:|------:|------:|------:|
Training (usec) | 577 | 733 | 1160
Prediction Train (usec) | 236 | 301 | 374
F1 Train | 0.855 | 0.8321 | 0.8088
Prediction Test (usec) | 417 | 268 | 374
F1 Test | 0.7481 | 0.7132 | 0.75


### Random Forest

Random Forests are an ensemble of Decision Trees. The goal of ensemble methods is to combine the predictions of several base estimators built with a given learning algorithm in order to improve generalizability / robustness over a single estimator.

Time Complexity: 

> O(M sqrt(F) N logN)

Space Complexity: 

> O(sqrt(F) N logN)

M = Number of trees,
N = Number of training samples,
F = Number of features,
and assuming best pratice that max_features = sqrt(F)

Tradeoffs*:

In random forests, each tree in the ensemble is built from a sample drawn with replacement (i.e., a bootstrap sample) from the training set. In addition, when splitting a node during the construction of the tree, the split that is chosen is no longer the best split among all features. Instead, the split that is picked is the best split among a random subset of the features. As a result of this randomness, the bias of the forest usually slightly increases (with respect to the bias of a single non-random tree) but, due to averaging, its variance also decreases, usually more than compensating for the increase in bias, hence yielding an overall better model.

Results:

Training Set Size | 100 | 200 | 300
------:|------:|------:|------:|
Training (usec) | 19311 | 20757 | 27922
Prediction Train (usec) | 818 | 965 | 1101
F1 Train | 0.9841 | 1.0 | 0.9976
Prediction Test (usec) | 738 | 760 | 880
F1 Test | 0.7119 | 0.7761 | 0.7344

*source scikit-learn

In [48]:
# Train a model
import time
from datetime import datetime

def train_classifier(clf, X_train, y_train):
    print "Training {}...".format(clf.__class__.__name__)
    #start = time.time()
    start = datetime.now()
    clf.fit(X_train, y_train)
    end = datetime.now()
    print "Done!\nTraining time (usec): {:.0f}".format(end.microsecond - start.microsecond)

# TODO: Choose a model, import it and instantiate an object
from sklearn import tree 
clf_dt = tree.DecisionTreeClassifier(random_state=seed)

# Fit model to training data
train_classifier(clf_dt, X_train, y_train)  # note: using entire training set here
#print clf_dt.get_params()  # you can inspect the learned model by printing it

print "Depth of decision tree: %s" % clf_dt.tree_.max_depth
    

Training DecisionTreeClassifier...
Done!
Training time (usec): 2073
Depth of decision tree: 14


In [49]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score,accuracy_score

def predict_labels(clf, features, target):
    print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = datetime.now()
    y_pred = clf.predict(features)
    end = datetime.now()
    print "Done!\nPrediction time (usec): {:.0f}".format(end.microsecond - start.microsecond)
    #print "Accuracy score: ({})".format(accuracy_score(target.values, y_pred))
    return f1_score(target.values, y_pred, pos_label='yes')

train_f1_score = predict_labels(clf_dt, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (usec): 576
F1 score for training set: 1.0


In [50]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf_dt, X_test, y_test))

Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (usec): 425
F1 score for test set: 0.716666666667


In [51]:
# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test):
    print "------------------------------------------"
    print "Training set size: %s" % X_train.shape[0]
    print "Test set size: %s" % X_test.shape[0]
    train_classifier(clf, X_train, y_train)
    print "F1 score for training set: {}".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

# TODO: Run the helper function above for desired subsets of training data
# Note: Keep the test set constant

train_predict(clf_dt, X_train, y_train, X_test, y_test)

train_predict(clf_dt, X_train[:200], y_train[:200], X_test, y_test)

train_predict(clf_dt, X_train[:100], y_train[:100], X_test, y_test)

------------------------------------------
Training set size: 300
Test set size: 95
Training DecisionTreeClassifier...
Done!
Training time (usec): 2014
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (usec): 260
F1 score for training set: 1.0
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (usec): 164
F1 score for test set: 0.716666666667
------------------------------------------
Training set size: 200
Test set size: 95
Training DecisionTreeClassifier...
Done!
Training time (usec): 1162
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (usec): 170
F1 score for training set: 1.0
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (usec): 154
F1 score for test set: 0.713178294574
------------------------------------------
Training set size: 100
Test set size: 95
Training DecisionTreeClassifier...
Done!
Training time (usec): 637
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time

In [52]:
# TODO: Train and predict using two other models
from sklearn import svm
clf_svc = svm.SVC()

train_predict(clf_svc, X_train, y_train, X_test, y_test)

train_predict(clf_svc, X_train[:200], y_train[:200], X_test, y_test)

train_predict(clf_svc, X_train[:100], y_train[:100], X_test, y_test)

------------------------------------------
Training set size: 300
Test set size: 95
Training SVC...
Done!
Training time (usec): 5678
Predicting labels using SVC...
Done!
Prediction time (usec): 4011
F1 score for training set: 0.869198312236
Predicting labels using SVC...
Done!
Prediction time (usec): 1497
F1 score for test set: 0.758620689655
------------------------------------------
Training set size: 200
Test set size: 95
Training SVC...
Done!
Training time (usec): 2814
Predicting labels using SVC...
Done!
Prediction time (usec): 2094
F1 score for training set: 0.869281045752
Predicting labels using SVC...
Done!
Prediction time (usec): 1047
F1 score for test set: 0.775510204082
------------------------------------------
Training set size: 100
Test set size: 95
Training SVC...
Done!
Training time (usec): 950
Predicting labels using SVC...
Done!
Prediction time (usec): 686
F1 score for training set: 0.859060402685
Predicting labels using SVC...
Done!
Prediction time (usec): 606
F1 sco

In [53]:
from sklearn.naive_bayes import GaussianNB

clf_nb = GaussianNB()

train_predict(clf_nb, X_train, y_train, X_test, y_test)

train_predict(clf_nb, X_train[:200], y_train[:200], X_test, y_test)

train_predict(clf_nb, X_train[:100], y_train[:100], X_test, y_test)

------------------------------------------
Training set size: 300
Test set size: 95
Training GaussianNB...
Done!
Training time (usec): 1636
Predicting labels using GaussianNB...
Done!
Prediction time (usec): 477
F1 score for training set: 0.808823529412
Predicting labels using GaussianNB...
Done!
Prediction time (usec): 282
F1 score for test set: 0.75
------------------------------------------
Training set size: 200
Test set size: 95
Training GaussianNB...
Done!
Training time (usec): 636
Predicting labels using GaussianNB...
Done!
Prediction time (usec): 298
F1 score for training set: 0.832061068702
Predicting labels using GaussianNB...
Done!
Prediction time (usec): 238
F1 score for test set: 0.713178294574
------------------------------------------
Training set size: 100
Test set size: 95
Training GaussianNB...
Done!
Training time (usec): 620
Predicting labels using GaussianNB...
Done!
Prediction time (usec): 269
F1 score for training set: 0.854961832061
Predicting labels using Gaussi

In [54]:
from sklearn.ensemble import RandomForestClassifier

clf_rf = RandomForestClassifier(random_state=seed)

train_predict(clf_rf, X_train, y_train, X_test, y_test)

train_predict(clf_rf, X_train[:200], y_train[:200], X_test, y_test)

train_predict(clf_rf, X_train[:100], y_train[:100], X_test, y_test)

------------------------------------------
Training set size: 300
Test set size: 95
Training RandomForestClassifier...
Done!
Training time (usec): 26280
Predicting labels using RandomForestClassifier...
Done!
Prediction time (usec): 1261
F1 score for training set: 0.997578692494
Predicting labels using RandomForestClassifier...
Done!
Prediction time (usec): 925
F1 score for test set: 0.734375
------------------------------------------
Training set size: 200
Test set size: 95
Training RandomForestClassifier...
Done!
Training time (usec): 24551
Predicting labels using RandomForestClassifier...
Done!
Prediction time (usec): 1088
F1 score for training set: 1.0
Predicting labels using RandomForestClassifier...
Done!
Prediction time (usec): 852
F1 score for test set: 0.776119402985
------------------------------------------
Training set size: 100
Test set size: 95
Training RandomForestClassifier...
Done!
Training time (usec): 20076
Predicting labels using RandomForestClassifier...
Done!
Pred

## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?
- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).
- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
- What is the model's final F<sub>1</sub> score?

----
#### What model to choose:

In terms of accuracy the results rank like this after grid search (F1 score):

Random Forest (0.8112) > Decision Tree (0.7914) > SVM (0.7862). 

For comparison NB F1 is 0.75 (no grid search).

Given accuracy results I'd drop NB. Given that SVM is higher in compuational complexity compared to Decision Tree (see results section 4), I'd drop SVM. Another drawback of SVMs is their black box nature. That means results cannot be easily interpreted.

That leaves Decision Trees and Random Forests. Random Forests are about 2% more accurate. But they do have higher computational complexity M * sqrt(F) > F. They also make interpretation of results a bit harder.

In summary, I'd choose Decision Trees. They are fast, easy to interpret and sufficiently accurate.

#### How it works:

A tree can be learned by splitting the source set into subsets based on a feature value test, e.g information gain. This process is repeated on each derived subset in a recursive manner. The recursion is completed when the subset at a node has all the same value of the target variable, or when splitting no longer adds value to the predictions. This process of top-down induction of decision trees is an example of a greedy algorithm, and it is by far the most common strategy for learning decision trees from data.*

*wikipedia, mostly

#### Tuning Results:

The Decision Tree grid search returned these as best parameters to choose:

Best params: {'criterion': 'entropy', 'max_depth': 4}

With a final F1 score of 0.7914.


In [55]:
# TODO: Fine-tune your model and report the best F1 score
from sklearn import grid_search

print "Grid search SVC ..."
parameters = {'kernel':('linear','rbf'), 'C':[1,2,3,5,10], 'gamma':[0.001]}
clf_svc_grd = grid_search.GridSearchCV(svm.SVC(), parameters, cv=3)
clf_svc_grd.fit(X_train,y_train)

# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf_svc_grd, X_test, y_test))
print "Best params: %s" % clf_svc_grd.best_params_
print

print "Grid search Decision Tree ..."
parameters = {'max_depth':(4,5,6,7,8,9,10),'criterion':('gini','entropy')}
clf_dt_grd = grid_search.GridSearchCV(tree.DecisionTreeClassifier(random_state=seed),parameters, cv=6)
clf_dt_grd.fit(X_train, y_train)

# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf_dt_grd, X_test, y_test))
print "Best params: %s" % clf_dt_grd.best_params_
print

# best practice choose max_features = sqrt(n_features)
print "Grid search Random Forest ..."
parameters = {'n_estimators':(8,9,10,11,12),'max_features':(5,6,7,8,9)}
clf_rf_grd = grid_search.GridSearchCV(RandomForestClassifier(random_state=seed),parameters, cv=6)
clf_rf_grd.fit(X_train, y_train)

# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf_rf_grd, X_test, y_test))
print "Best params: %s" % clf_rf_grd.best_params_

Grid search SVC ...
Predicting labels using GridSearchCV...
Done!
Prediction time (usec): 1329
F1 score for test set: 0.786206896552
Best params: {'kernel': 'rbf', 'C': 10, 'gamma': 0.001}

Grid search Decision Tree ...
Predicting labels using GridSearchCV...
Done!
Prediction time (usec): 176
F1 score for test set: 0.791366906475
Best params: {'criterion': 'entropy', 'max_depth': 4}

Grid search Random Forest ...
Predicting labels using GridSearchCV...
Done!
Prediction time (usec): 1395
F1 score for test set: 0.811188811189
Best params: {'max_features': 8, 'n_estimators': 11}
